In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [7]:
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)

selector= RFE(estimator= lr, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1125
           1       0.35      0.49      0.41       628

    accuracy                           0.49      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.3493150684931507
Accuracy: 0.49115801483171706
AUC: 0.4822179759377212


--- DecisionTreeClassifier ---

In [8]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)

selector= RFE(estimator= dt, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.65      0.65      1125
           1       0.37      0.37      0.37       628

    accuracy                           0.55      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.55      0.55      1753

Precision: 0.3682539682539683
Accuracy: 0.547062179121506
AUC: 0.5078244869072894


--- Tuned - DecisionTreeClassifier ---

In [16]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best Precision Score: 0.6250674694419338


In [18]:
tuned_dt= gs_dt.best_estimator_

selector= RFE(estimator= tuned_dt, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.64      0.91      0.76      1125
           1       0.39      0.10      0.16       628

    accuracy                           0.62      1753
   macro avg       0.52      0.51      0.46      1753
weighted avg       0.55      0.62      0.54      1753

Precision: 0.3888888888888889
Accuracy: 0.6212207644038791
AUC: 0.5058414720452937


--- Random Forest ---

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state= 15)

selector= RFE(estimator= rf, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [27]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.83      0.73      1125
           1       0.37      0.18      0.24       628

    accuracy                           0.60      1753
   macro avg       0.51      0.50      0.48      1753
weighted avg       0.55      0.60      0.55      1753

Precision: 0.3708609271523179
Accuracy: 0.5972618368511123
AUC: 0.5257133757961784


In [28]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.8523196063573331


In [29]:
tuned_rf= gs_tuned_rf.best_estimator_

selector= RFE(estimator= tuned_rf, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.65      0.86      0.74      1125
           1       0.39      0.16      0.22       628

    accuracy                           0.61      1753
   macro avg       0.52      0.51      0.48      1753
weighted avg       0.55      0.61      0.55      1753

Precision: 0.3858267716535433
Accuracy: 0.6086708499714775
AUC: 0.5258917197452229


--- AdaBoost ---

In [30]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)

selector= RFE(estimator= ada, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [31]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.64      0.74      0.69      1125
           1       0.36      0.26      0.30       628

    accuracy                           0.57      1753
   macro avg       0.50      0.50      0.50      1753
weighted avg       0.54      0.57      0.55      1753

Precision: 0.36283185840707965
Accuracy: 0.5710211066742726
AUC: 0.5007756546355272


In [32]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6590501578274092


In [33]:
tuned_ada= gs_ada.best_estimator_

selector= RFE(estimator= tuned_ada, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.64      0.99      0.78      1125
           1       0.40      0.02      0.03       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.40      1753
weighted avg       0.56      0.64      0.51      1753

Precision: 0.4
Accuracy: 0.638904734740445
AUC: 0.4938542108987969


--- GradientBoosting ---

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)

selector= RFE(estimator= grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.97      0.77      1125
           1       0.41      0.03      0.06       628

    accuracy                           0.64      1753
   macro avg       0.53      0.50      0.42      1753
weighted avg       0.56      0.64      0.52      1753

Precision: 0.4117647058823529
Accuracy: 0.6366229321163719
AUC: 0.5132448690728946


In [35]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7257639977763283


In [36]:
tuned_grb= gs_grb.best_estimator_

selector= RFE(estimator= tuned_grb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.95      0.76      1125
           1       0.33      0.05      0.08       628

    accuracy                           0.62      1753
   macro avg       0.48      0.50      0.42      1753
weighted avg       0.53      0.62      0.52      1753

Precision: 0.3258426966292135
Accuracy: 0.6240730176839704
AUC: 0.5161910828025478


--- XGB ---

In [37]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)

selector= RFE(estimator= xgb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.65      0.79      0.71      1125
           1       0.37      0.22      0.28       628

    accuracy                           0.59      1753
   macro avg       0.51      0.51      0.49      1753
weighted avg       0.55      0.59      0.56      1753

Precision: 0.3726541554959786
Accuracy: 0.587564175698802
AUC: 0.5141033262561925


In [38]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_

selector= RFE(estimator= tuned_xgb, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7257272612827685
              precision    recall  f1-score   support

           0       0.64      0.91      0.75      1125
           1       0.36      0.09      0.14       628

    accuracy                           0.62      1753
   macro avg       0.50      0.50      0.45      1753
weighted avg       0.54      0.62      0.54      1753

Precision: 0.36363636363636365
Accuracy: 0.6177980604677695
AUC: 0.5061755130927106


--- LGBM ---

In [40]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)

selector= RFE(estimator= lgm, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]


lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000536 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4911
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4907
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, t

In [41]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_

selector= RFE(estimator= tuned_lgm, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000351 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4911
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.8482220142848164
[LightGBM] [Warning] min_data_in_leaf is set=30, min

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000393 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2548
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightG

--- CatBoost ---

In [42]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)

selector= RFE(estimator= cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.02579
0:	learn: 0.6889201	total: 141ms	remaining: 2m 20s
1:	learn: 0.6859547	total: 147ms	remaining: 1m 13s
2:	learn: 0.6837306	total: 153ms	remaining: 50.7s
3:	learn: 0.6820321	total: 158ms	remaining: 39.3s
4:	learn: 0.6803403	total: 163ms	remaining: 32.5s
5:	learn: 0.6782150	total: 168ms	remaining: 27.9s
6:	learn: 0.6770789	total: 174ms	remaining: 24.7s
7:	learn: 0.6728095	total: 179ms	remaining: 22.2s
8:	learn: 0.6697819	total: 186ms	remaining: 20.5s
9:	learn: 0.6681794	total: 191ms	remaining: 18.9s
10:	learn: 0.6668213	total: 198ms	remaining: 17.8s
11:	learn: 0.6645231	total: 203ms	remaining: 16.7s
12:	learn: 0.6621362	total: 208ms	remaining: 15.8s
13:	learn: 0.6586652	total: 213ms	remaining: 15s
14:	learn: 0.6560947	total: 218ms	remaining: 14.3s
15:	learn: 0.6539516	total: 223ms	remaining: 13.7s
16:	learn: 0.6514640	total: 228ms	remaining: 13.2s
17:	learn: 0.6488185	total: 233ms	remaining: 12.7s
18:	learn: 0.6477980	total: 238ms	remaining: 12.3s
19:	learn: 0

178:	learn: 0.5295083	total: 1.08s	remaining: 4.95s
179:	learn: 0.5292100	total: 1.08s	remaining: 4.94s
180:	learn: 0.5289981	total: 1.09s	remaining: 4.93s
181:	learn: 0.5286645	total: 1.09s	remaining: 4.92s
182:	learn: 0.5281952	total: 1.1s	remaining: 4.91s
183:	learn: 0.5275800	total: 1.1s	remaining: 4.9s
184:	learn: 0.5273399	total: 1.11s	remaining: 4.89s
185:	learn: 0.5269993	total: 1.12s	remaining: 4.88s
186:	learn: 0.5266592	total: 1.12s	remaining: 4.88s
187:	learn: 0.5263101	total: 1.13s	remaining: 4.87s
188:	learn: 0.5261075	total: 1.13s	remaining: 4.87s
189:	learn: 0.5258317	total: 1.14s	remaining: 4.86s
190:	learn: 0.5254608	total: 1.15s	remaining: 4.85s
191:	learn: 0.5252352	total: 1.15s	remaining: 4.85s
192:	learn: 0.5250543	total: 1.16s	remaining: 4.84s
193:	learn: 0.5248070	total: 1.16s	remaining: 4.83s
194:	learn: 0.5246172	total: 1.17s	remaining: 4.83s
195:	learn: 0.5241696	total: 1.17s	remaining: 4.82s
196:	learn: 0.5239819	total: 1.18s	remaining: 4.81s
197:	learn: 0.5

350:	learn: 0.4850516	total: 1.98s	remaining: 3.66s
351:	learn: 0.4847142	total: 1.99s	remaining: 3.66s
352:	learn: 0.4844899	total: 1.99s	remaining: 3.65s
353:	learn: 0.4840514	total: 2s	remaining: 3.64s
354:	learn: 0.4838497	total: 2s	remaining: 3.64s
355:	learn: 0.4837110	total: 2.01s	remaining: 3.63s
356:	learn: 0.4833061	total: 2.01s	remaining: 3.62s
357:	learn: 0.4830626	total: 2.02s	remaining: 3.62s
358:	learn: 0.4827879	total: 2.02s	remaining: 3.61s
359:	learn: 0.4825493	total: 2.03s	remaining: 3.6s
360:	learn: 0.4822907	total: 2.03s	remaining: 3.6s
361:	learn: 0.4819843	total: 2.04s	remaining: 3.59s
362:	learn: 0.4817334	total: 2.04s	remaining: 3.58s
363:	learn: 0.4813606	total: 2.05s	remaining: 3.58s
364:	learn: 0.4811032	total: 2.05s	remaining: 3.57s
365:	learn: 0.4808809	total: 2.06s	remaining: 3.56s
366:	learn: 0.4807051	total: 2.06s	remaining: 3.56s
367:	learn: 0.4804259	total: 2.07s	remaining: 3.55s
368:	learn: 0.4801037	total: 2.07s	remaining: 3.54s
369:	learn: 0.479842

521:	learn: 0.4419645	total: 2.88s	remaining: 2.64s
522:	learn: 0.4417390	total: 2.89s	remaining: 2.63s
523:	learn: 0.4415758	total: 2.89s	remaining: 2.63s
524:	learn: 0.4413256	total: 2.9s	remaining: 2.62s
525:	learn: 0.4410119	total: 2.9s	remaining: 2.62s
526:	learn: 0.4407593	total: 2.91s	remaining: 2.61s
527:	learn: 0.4405031	total: 2.91s	remaining: 2.6s
528:	learn: 0.4402653	total: 2.92s	remaining: 2.6s
529:	learn: 0.4400249	total: 2.92s	remaining: 2.59s
530:	learn: 0.4398397	total: 2.93s	remaining: 2.59s
531:	learn: 0.4395734	total: 2.94s	remaining: 2.58s
532:	learn: 0.4393402	total: 2.94s	remaining: 2.58s
533:	learn: 0.4390547	total: 2.95s	remaining: 2.57s
534:	learn: 0.4387932	total: 2.95s	remaining: 2.57s
535:	learn: 0.4385173	total: 2.96s	remaining: 2.56s
536:	learn: 0.4383157	total: 2.96s	remaining: 2.55s
537:	learn: 0.4381207	total: 2.97s	remaining: 2.55s
538:	learn: 0.4378424	total: 2.97s	remaining: 2.54s
539:	learn: 0.4376562	total: 2.98s	remaining: 2.54s
540:	learn: 0.43

686:	learn: 0.4075949	total: 3.78s	remaining: 1.72s
687:	learn: 0.4073436	total: 3.79s	remaining: 1.72s
688:	learn: 0.4071672	total: 3.79s	remaining: 1.71s
689:	learn: 0.4070067	total: 3.8s	remaining: 1.71s
690:	learn: 0.4068051	total: 3.8s	remaining: 1.7s
691:	learn: 0.4065972	total: 3.81s	remaining: 1.69s
692:	learn: 0.4063737	total: 3.81s	remaining: 1.69s
693:	learn: 0.4062027	total: 3.82s	remaining: 1.68s
694:	learn: 0.4059833	total: 3.82s	remaining: 1.68s
695:	learn: 0.4058189	total: 3.83s	remaining: 1.67s
696:	learn: 0.4056353	total: 3.83s	remaining: 1.67s
697:	learn: 0.4054288	total: 3.84s	remaining: 1.66s
698:	learn: 0.4051970	total: 3.84s	remaining: 1.65s
699:	learn: 0.4050328	total: 3.85s	remaining: 1.65s
700:	learn: 0.4047933	total: 3.85s	remaining: 1.64s
701:	learn: 0.4046095	total: 3.86s	remaining: 1.64s
702:	learn: 0.4044586	total: 3.86s	remaining: 1.63s
703:	learn: 0.4043327	total: 3.87s	remaining: 1.63s
704:	learn: 0.4041128	total: 3.87s	remaining: 1.62s
705:	learn: 0.4

...

76:	learn: 0.5698116	total: 356ms	remaining: 4.27s
77:	learn: 0.5690483	total: 361ms	remaining: 4.27s
78:	learn: 0.5686425	total: 366ms	remaining: 4.27s
79:	learn: 0.5676221	total: 371ms	remaining: 4.27s
80:	learn: 0.5670060	total: 376ms	remaining: 4.26s
81:	learn: 0.5666370	total: 380ms	remaining: 4.26s
82:	learn: 0.5662684	total: 385ms	remaining: 4.25s
83:	learn: 0.5660276	total: 389ms	remaining: 4.24s
84:	learn: 0.5651997	total: 394ms	remaining: 4.24s
85:	learn: 0.5646959	total: 398ms	remaining: 4.23s
86:	learn: 0.5638374	total: 403ms	remaining: 4.23s
87:	learn: 0.5632791	total: 407ms	remaining: 4.22s
88:	learn: 0.5628168	total: 412ms	remaining: 4.22s
89:	learn: 0.5619690	total: 417ms	remaining: 4.21s
90:	learn: 0.5616004	total: 422ms	remaining: 4.21s
91:	learn: 0.5613675	total: 426ms	remaining: 4.21s
92:	learn: 0.5608049	total: 431ms	remaining: 4.2s
93:	learn: 0.5604697	total: 435ms	remaining: 4.19s
94:	learn: 0.5601372	total: 440ms	remaining: 4.19s
95:	learn: 0.5598476	total: 445m

263:	learn: 0.5090808	total: 1.25s	remaining: 3.48s
264:	learn: 0.5089008	total: 1.25s	remaining: 3.47s
265:	learn: 0.5085928	total: 1.26s	remaining: 3.47s
266:	learn: 0.5084087	total: 1.26s	remaining: 3.46s
267:	learn: 0.5082047	total: 1.27s	remaining: 3.46s
268:	learn: 0.5080797	total: 1.27s	remaining: 3.45s
269:	learn: 0.5079651	total: 1.27s	remaining: 3.45s
270:	learn: 0.5077510	total: 1.28s	remaining: 3.44s
271:	learn: 0.5075631	total: 1.28s	remaining: 3.44s
272:	learn: 0.5073364	total: 1.29s	remaining: 3.43s
273:	learn: 0.5072361	total: 1.29s	remaining: 3.43s
274:	learn: 0.5070294	total: 1.3s	remaining: 3.42s
275:	learn: 0.5069582	total: 1.3s	remaining: 3.42s
276:	learn: 0.5067045	total: 1.31s	remaining: 3.41s
277:	learn: 0.5065509	total: 1.31s	remaining: 3.41s
278:	learn: 0.5063569	total: 1.32s	remaining: 3.41s
279:	learn: 0.5061805	total: 1.32s	remaining: 3.4s
280:	learn: 0.5060297	total: 1.33s	remaining: 3.4s
281:	learn: 0.5058327	total: 1.33s	remaining: 3.39s
282:	learn: 0.50

452:	learn: 0.4721286	total: 2.14s	remaining: 2.59s
453:	learn: 0.4719595	total: 2.15s	remaining: 2.58s
454:	learn: 0.4717936	total: 2.15s	remaining: 2.58s
455:	learn: 0.4716431	total: 2.16s	remaining: 2.57s
456:	learn: 0.4715116	total: 2.16s	remaining: 2.57s
457:	learn: 0.4712196	total: 2.17s	remaining: 2.56s
458:	learn: 0.4710900	total: 2.17s	remaining: 2.56s
459:	learn: 0.4709164	total: 2.18s	remaining: 2.56s
460:	learn: 0.4707832	total: 2.18s	remaining: 2.55s
461:	learn: 0.4705969	total: 2.19s	remaining: 2.55s
462:	learn: 0.4704441	total: 2.19s	remaining: 2.54s
463:	learn: 0.4702995	total: 2.2s	remaining: 2.54s
464:	learn: 0.4701219	total: 2.2s	remaining: 2.53s
465:	learn: 0.4699613	total: 2.21s	remaining: 2.53s
466:	learn: 0.4698304	total: 2.21s	remaining: 2.52s
467:	learn: 0.4696245	total: 2.21s	remaining: 2.52s
468:	learn: 0.4693260	total: 2.22s	remaining: 2.51s
469:	learn: 0.4692016	total: 2.22s	remaining: 2.51s
470:	learn: 0.4690518	total: 2.23s	remaining: 2.5s
471:	learn: 0.4

645:	learn: 0.4406491	total: 3.04s	remaining: 1.67s
646:	learn: 0.4405404	total: 3.04s	remaining: 1.66s
647:	learn: 0.4404072	total: 3.05s	remaining: 1.66s
648:	learn: 0.4403091	total: 3.05s	remaining: 1.65s
649:	learn: 0.4401539	total: 3.06s	remaining: 1.65s
650:	learn: 0.4400228	total: 3.06s	remaining: 1.64s
651:	learn: 0.4398612	total: 3.07s	remaining: 1.64s
652:	learn: 0.4396915	total: 3.07s	remaining: 1.63s
653:	learn: 0.4395950	total: 3.08s	remaining: 1.63s
654:	learn: 0.4394488	total: 3.08s	remaining: 1.62s
655:	learn: 0.4392909	total: 3.09s	remaining: 1.62s
656:	learn: 0.4391678	total: 3.09s	remaining: 1.61s
657:	learn: 0.4390163	total: 3.1s	remaining: 1.61s
658:	learn: 0.4388560	total: 3.1s	remaining: 1.6s
659:	learn: 0.4387265	total: 3.1s	remaining: 1.6s
660:	learn: 0.4385428	total: 3.11s	remaining: 1.59s
661:	learn: 0.4384397	total: 3.11s	remaining: 1.59s
662:	learn: 0.4382724	total: 3.12s	remaining: 1.58s
663:	learn: 0.4380952	total: 3.12s	remaining: 1.58s
664:	learn: 0.437

829:	learn: 0.4167683	total: 3.93s	remaining: 805ms
830:	learn: 0.4166597	total: 3.93s	remaining: 800ms
831:	learn: 0.4165110	total: 3.94s	remaining: 795ms
832:	learn: 0.4163899	total: 3.94s	remaining: 791ms
833:	learn: 0.4162880	total: 3.95s	remaining: 786ms
834:	learn: 0.4161329	total: 3.96s	remaining: 782ms
835:	learn: 0.4160043	total: 3.96s	remaining: 777ms
836:	learn: 0.4159073	total: 3.96s	remaining: 772ms
837:	learn: 0.4157450	total: 3.97s	remaining: 768ms
838:	learn: 0.4156186	total: 3.98s	remaining: 763ms
839:	learn: 0.4155161	total: 3.98s	remaining: 758ms
840:	learn: 0.4154009	total: 3.99s	remaining: 754ms
841:	learn: 0.4153324	total: 3.99s	remaining: 749ms
842:	learn: 0.4152085	total: 4s	remaining: 744ms
843:	learn: 0.4151450	total: 4s	remaining: 740ms
844:	learn: 0.4150490	total: 4.01s	remaining: 735ms
845:	learn: 0.4149323	total: 4.01s	remaining: 730ms
846:	learn: 0.4148177	total: 4.02s	remaining: 726ms
847:	learn: 0.4147263	total: 4.02s	remaining: 721ms
848:	learn: 0.4146

In [43]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_

selector= RFE(estimator= tuned_cat, n_features_to_select=10)
selector= selector.fit(X_sm_tl, y_sm_tl)
selected_features= selector.support_

X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6907298	total: 45.1ms	remaining: 4.46s
1:	learn: 0.6894892	total: 87.7ms	remaining: 4.29s
2:	learn: 0.6885905	total: 131ms	remaining: 4.25s
3:	learn: 0.6869870	total: 175ms	remaining: 4.2s
4:	learn: 0.6853325	total: 221ms	remaining: 4.2s
5:	learn: 0.6834098	total: 270ms	remaining: 4.23s
6:	learn: 0.6814347	total: 318ms	remaining: 4.23s
7:	learn: 0.6801403	total: 367ms	remaining: 4.22s
8:	learn: 0.6788242	total: 416ms	remaining: 4.21s
9:	learn: 0.6776116	total: 465ms	remaining: 4.19s
10:	learn: 0.6759184	total: 518ms	remaining: 4.19s
11:	learn: 0.6746354	total: 569ms	remaining: 4.18s
12:	learn: 0.6723875	total: 620ms	remaining: 4.15s
13:	learn: 0.6711261	total: 673ms	remaining: 4.13s
14:	learn: 0.6698886	total: 722ms	remaining: 4.09s
15:	learn: 0.6686721	total: 774ms	remaining: 4.07s
16:	learn: 0.6673800	total: 825ms	remaining: 4.03s
17:	learn: 0.6663922	total: 875ms	remaining: 3.99s
18:	learn: 0.6655287	total: 92

61:	learn: 0.6199643	total: 3.26s	remaining: 2s
62:	learn: 0.6190781	total: 3.32s	remaining: 1.95s
63:	learn: 0.6181662	total: 3.37s	remaining: 1.89s
64:	learn: 0.6174932	total: 3.42s	remaining: 1.84s
65:	learn: 0.6167251	total: 3.47s	remaining: 1.79s
66:	learn: 0.6158252	total: 3.53s	remaining: 1.74s
67:	learn: 0.6150693	total: 3.58s	remaining: 1.68s
68:	learn: 0.6144151	total: 3.63s	remaining: 1.63s
69:	learn: 0.6135373	total: 3.69s	remaining: 1.58s
70:	learn: 0.6128884	total: 3.74s	remaining: 1.53s
71:	learn: 0.6120146	total: 3.79s	remaining: 1.47s
72:	learn: 0.6111377	total: 3.84s	remaining: 1.42s
73:	learn: 0.6103082	total: 3.89s	remaining: 1.37s
74:	learn: 0.6094837	total: 3.95s	remaining: 1.31s
75:	learn: 0.6086647	total: 4s	remaining: 1.26s
76:	learn: 0.6080368	total: 4.05s	remaining: 1.21s
77:	learn: 0.6069506	total: 4.1s	remaining: 1.16s
78:	learn: 0.6061362	total: 4.16s	remaining: 1.1s
79:	learn: 0.6054403	total: 4.21s	remaining: 1.05s
80:	learn: 0.6047292	total: 4.26s	remai

24:	learn: 0.6597358	total: 1.25s	remaining: 3.75s
25:	learn: 0.6589805	total: 1.3s	remaining: 3.69s
26:	learn: 0.6571112	total: 1.35s	remaining: 3.64s
27:	learn: 0.6559695	total: 1.4s	remaining: 3.59s
28:	learn: 0.6546849	total: 1.44s	remaining: 3.54s
29:	learn: 0.6537753	total: 1.5s	remaining: 3.49s
30:	learn: 0.6516994	total: 1.54s	remaining: 3.43s
31:	learn: 0.6506954	total: 1.59s	remaining: 3.38s
32:	learn: 0.6499035	total: 1.64s	remaining: 3.33s
33:	learn: 0.6489765	total: 1.69s	remaining: 3.28s
34:	learn: 0.6477775	total: 1.74s	remaining: 3.23s
35:	learn: 0.6465322	total: 1.79s	remaining: 3.19s
36:	learn: 0.6456634	total: 1.85s	remaining: 3.15s
37:	learn: 0.6447218	total: 1.91s	remaining: 3.12s
38:	learn: 0.6433480	total: 1.97s	remaining: 3.08s
39:	learn: 0.6422342	total: 2.02s	remaining: 3.03s
40:	learn: 0.6412040	total: 2.07s	remaining: 2.98s
41:	learn: 0.6401346	total: 2.12s	remaining: 2.93s
42:	learn: 0.6391221	total: 2.18s	remaining: 2.89s
43:	learn: 0.6378668	total: 2.24s	

90:	learn: 0.6482488	total: 4.53s	remaining: 448ms
91:	learn: 0.6476485	total: 4.57s	remaining: 398ms
92:	learn: 0.6472625	total: 4.62s	remaining: 348ms
93:	learn: 0.6468876	total: 4.67s	remaining: 298ms
94:	learn: 0.6465335	total: 4.71s	remaining: 248ms
95:	learn: 0.6461853	total: 4.76s	remaining: 198ms
96:	learn: 0.6457631	total: 4.81s	remaining: 149ms
97:	learn: 0.6451705	total: 4.85s	remaining: 99ms
98:	learn: 0.6448098	total: 4.9s	remaining: 49.5ms
99:	learn: 0.6444781	total: 4.94s	remaining: 0us
0:	learn: 0.6925873	total: 42.4ms	remaining: 4.2s
1:	learn: 0.6919838	total: 87.2ms	remaining: 4.27s
2:	learn: 0.6913460	total: 131ms	remaining: 4.23s
3:	learn: 0.6909759	total: 176ms	remaining: 4.23s
4:	learn: 0.6902458	total: 219ms	remaining: 4.17s
5:	learn: 0.6898193	total: 266ms	remaining: 4.17s
6:	learn: 0.6890084	total: 311ms	remaining: 4.13s
7:	learn: 0.6885494	total: 356ms	remaining: 4.09s
8:	learn: 0.6880577	total: 399ms	remaining: 4.04s
9:	learn: 0.6875990	total: 444ms	remaining

55:	learn: 0.6621924	total: 2.46s	remaining: 1.94s
56:	learn: 0.6613765	total: 2.51s	remaining: 1.89s
57:	learn: 0.6609518	total: 2.55s	remaining: 1.85s
58:	learn: 0.6605141	total: 2.59s	remaining: 1.8s
59:	learn: 0.6600328	total: 2.64s	remaining: 1.76s
60:	learn: 0.6596197	total: 2.68s	remaining: 1.72s
61:	learn: 0.6591431	total: 2.73s	remaining: 1.67s
62:	learn: 0.6587017	total: 2.77s	remaining: 1.63s
63:	learn: 0.6578819	total: 2.81s	remaining: 1.58s
64:	learn: 0.6573181	total: 2.86s	remaining: 1.54s
65:	learn: 0.6568552	total: 2.9s	remaining: 1.49s
66:	learn: 0.6562460	total: 2.94s	remaining: 1.45s
67:	learn: 0.6557926	total: 2.98s	remaining: 1.41s
68:	learn: 0.6554982	total: 3.03s	remaining: 1.36s
69:	learn: 0.6551732	total: 3.07s	remaining: 1.32s
70:	learn: 0.6547949	total: 3.12s	remaining: 1.27s
71:	learn: 0.6540330	total: 3.16s	remaining: 1.23s
72:	learn: 0.6536647	total: 3.2s	remaining: 1.18s
73:	learn: 0.6530566	total: 3.24s	remaining: 1.14s
74:	learn: 0.6526479	total: 3.29s	

19:	learn: 0.6810888	total: 728ms	remaining: 2.91s
20:	learn: 0.6804157	total: 767ms	remaining: 2.88s
21:	learn: 0.6798714	total: 802ms	remaining: 2.84s
22:	learn: 0.6793183	total: 838ms	remaining: 2.81s
23:	learn: 0.6784973	total: 873ms	remaining: 2.76s
24:	learn: 0.6778718	total: 908ms	remaining: 2.73s
25:	learn: 0.6774808	total: 946ms	remaining: 2.69s
26:	learn: 0.6771248	total: 982ms	remaining: 2.65s
27:	learn: 0.6766841	total: 1.02s	remaining: 2.62s
28:	learn: 0.6762325	total: 1.05s	remaining: 2.58s
29:	learn: 0.6755210	total: 1.09s	remaining: 2.54s
30:	learn: 0.6748630	total: 1.12s	remaining: 2.5s
31:	learn: 0.6744163	total: 1.16s	remaining: 2.46s
32:	learn: 0.6736197	total: 1.2s	remaining: 2.43s
33:	learn: 0.6732422	total: 1.23s	remaining: 2.39s
34:	learn: 0.6727332	total: 1.27s	remaining: 2.35s
35:	learn: 0.6721790	total: 1.3s	remaining: 2.32s
36:	learn: 0.6716414	total: 1.34s	remaining: 2.28s
37:	learn: 0.6709008	total: 1.38s	remaining: 2.24s
38:	learn: 0.6705296	total: 1.41s	

83:	learn: 0.6493128	total: 2.94s	remaining: 559ms
84:	learn: 0.6489828	total: 2.97s	remaining: 524ms
85:	learn: 0.6485623	total: 3s	remaining: 489ms
86:	learn: 0.6480920	total: 3.04s	remaining: 454ms
87:	learn: 0.6475069	total: 3.07s	remaining: 419ms
88:	learn: 0.6471583	total: 3.11s	remaining: 384ms
89:	learn: 0.6468682	total: 3.14s	remaining: 349ms
90:	learn: 0.6463558	total: 3.18s	remaining: 314ms
91:	learn: 0.6459730	total: 3.21s	remaining: 279ms
92:	learn: 0.6455372	total: 3.25s	remaining: 244ms
93:	learn: 0.6449410	total: 3.28s	remaining: 209ms
94:	learn: 0.6444134	total: 3.31s	remaining: 174ms
95:	learn: 0.6439426	total: 3.35s	remaining: 139ms
96:	learn: 0.6434269	total: 3.38s	remaining: 105ms
97:	learn: 0.6430813	total: 3.42s	remaining: 69.7ms
98:	learn: 0.6427397	total: 3.45s	remaining: 34.9ms
99:	learn: 0.6422504	total: 3.48s	remaining: 0us
0:	learn: 0.6926395	total: 32.9ms	remaining: 3.26s
1:	learn: 0.6919892	total: 66.7ms	remaining: 3.27s
2:	learn: 0.6912780	total: 101ms	r

45:	learn: 0.6650355	total: 1.46s	remaining: 1.72s
46:	learn: 0.6645004	total: 1.5s	remaining: 1.69s
47:	learn: 0.6639607	total: 1.53s	remaining: 1.65s
48:	learn: 0.6634786	total: 1.56s	remaining: 1.62s
49:	learn: 0.6630442	total: 1.59s	remaining: 1.59s
50:	learn: 0.6624770	total: 1.62s	remaining: 1.56s
51:	learn: 0.6619907	total: 1.65s	remaining: 1.52s
52:	learn: 0.6614535	total: 1.69s	remaining: 1.49s
53:	learn: 0.6609449	total: 1.72s	remaining: 1.46s
54:	learn: 0.6605941	total: 1.75s	remaining: 1.43s
55:	learn: 0.6600557	total: 1.78s	remaining: 1.4s
56:	learn: 0.6595162	total: 1.81s	remaining: 1.37s
57:	learn: 0.6591500	total: 1.84s	remaining: 1.33s
58:	learn: 0.6587745	total: 1.87s	remaining: 1.3s
59:	learn: 0.6583257	total: 1.91s	remaining: 1.27s
60:	learn: 0.6579290	total: 1.94s	remaining: 1.24s
61:	learn: 0.6575829	total: 1.98s	remaining: 1.21s
62:	learn: 0.6571338	total: 2.01s	remaining: 1.18s
63:	learn: 0.6566832	total: 2.04s	remaining: 1.15s
64:	learn: 0.6561482	total: 2.07s	

10:	learn: 0.6850148	total: 332ms	remaining: 2.69s
11:	learn: 0.6843668	total: 365ms	remaining: 2.68s
12:	learn: 0.6838457	total: 399ms	remaining: 2.67s
13:	learn: 0.6829845	total: 431ms	remaining: 2.65s
14:	learn: 0.6821658	total: 464ms	remaining: 2.63s
15:	learn: 0.6815477	total: 497ms	remaining: 2.61s
16:	learn: 0.6809532	total: 530ms	remaining: 2.59s
17:	learn: 0.6804239	total: 563ms	remaining: 2.56s
18:	learn: 0.6798962	total: 597ms	remaining: 2.54s
19:	learn: 0.6792169	total: 629ms	remaining: 2.51s
20:	learn: 0.6787732	total: 661ms	remaining: 2.49s
21:	learn: 0.6782033	total: 694ms	remaining: 2.46s
22:	learn: 0.6776585	total: 727ms	remaining: 2.43s
23:	learn: 0.6770710	total: 760ms	remaining: 2.41s
24:	learn: 0.6762137	total: 792ms	remaining: 2.37s
25:	learn: 0.6756065	total: 823ms	remaining: 2.34s
26:	learn: 0.6749738	total: 857ms	remaining: 2.32s
27:	learn: 0.6742367	total: 891ms	remaining: 2.29s
28:	learn: 0.6734820	total: 923ms	remaining: 2.26s
29:	learn: 0.6730129	total: 957

76:	learn: 0.6491541	total: 2.5s	remaining: 746ms
77:	learn: 0.6487676	total: 2.53s	remaining: 713ms
78:	learn: 0.6482679	total: 2.56s	remaining: 680ms
79:	learn: 0.6478564	total: 2.59s	remaining: 647ms
80:	learn: 0.6474515	total: 2.62s	remaining: 614ms
81:	learn: 0.6469713	total: 2.65s	remaining: 581ms
82:	learn: 0.6466601	total: 2.68s	remaining: 549ms
83:	learn: 0.6461782	total: 2.71s	remaining: 516ms
84:	learn: 0.6456950	total: 2.74s	remaining: 483ms
85:	learn: 0.6451924	total: 2.77s	remaining: 451ms
86:	learn: 0.6448427	total: 2.8s	remaining: 419ms
87:	learn: 0.6444556	total: 2.83s	remaining: 387ms
88:	learn: 0.6439184	total: 2.86s	remaining: 354ms
89:	learn: 0.6433866	total: 2.9s	remaining: 322ms
90:	learn: 0.6429136	total: 2.92s	remaining: 289ms
91:	learn: 0.6425614	total: 2.95s	remaining: 257ms
92:	learn: 0.6422461	total: 2.98s	remaining: 225ms
93:	learn: 0.6417519	total: 3.01s	remaining: 192ms
94:	learn: 0.6413607	total: 3.04s	remaining: 160ms
95:	learn: 0.6408907	total: 3.07s	

--- Visualisation ---

In [44]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

rfe_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
rfe_precision= rfe_precision.sort_values(by= 'Precision Score', ascending=False)
print(rfe_precision)

                                Model  Precision Score
14          Tuned CatBoost Precision:         1.000000
7         GradientBoosting Precision:         0.411765
6           Tuned AdaBoost Precision:         0.400000
2      Tuned Decision Tree Precision:         0.388889
4      Tuned Random Forest Precision:         0.385827
11                    LGBM Precision:         0.383333
9                      XGB Precision:         0.372654
3            Random Forest Precision:         0.370861
1            Decision Tree Precision:         0.368254
10               Tuned XGB Precision:         0.363636
5                 AdaBoost Precision:         0.362832
13                CatBoost Precision:         0.359477
0      Logistic Regression Precision:         0.349315
8   Tuned GradientBoosting Precision:         0.325843
12              Tuned LGBM Precision:         0.297297


In [45]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

rfe_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
rfe_accuracy= rfe_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(rfe_accuracy)

                               Model  Accuracy Score
14          Tuned CatBoost Accuracy:        0.642327
6           Tuned AdaBoost Accuracy:        0.638905
7         GradientBoosting Accuracy:        0.636623
12              Tuned LGBM Accuracy:        0.633200
8   Tuned GradientBoosting Accuracy:        0.624073
2      Tuned Decision Tree Accuracy:        0.621221
10               Tuned XGB Accuracy:        0.617798
13                CatBoost Accuracy:        0.617228
11                    LGBM Accuracy:        0.609812
4      Tuned Random Forest Accuracy:        0.608671
3            Random Forest Accuracy:        0.597262
9                      XGB Accuracy:        0.587564
5                 AdaBoost Accuracy:        0.571021
1            Decision Tree Accuracy:        0.547062
0      Logistic Regression Accuracy:        0.491158


In [46]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

rfe_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
rfe_auc= rfe_auc.sort_values(by= 'AUC Score', ascending=False)
print(rfe_auc)

                          Model  AUC Score
4      Tuned Random Forest AUC:   0.525892
3            Random Forest AUC:   0.525713
11                    LGBM AUC:   0.522290
8   Tuned GradientBoosting AUC:   0.516191
13                CatBoost AUC:   0.515837
9                      XGB AUC:   0.514103
7         GradientBoosting AUC:   0.513245
1            Decision Tree AUC:   0.507824
10               Tuned XGB AUC:   0.506176
2      Tuned Decision Tree AUC:   0.505841
12              Tuned LGBM AUC:   0.503063
5                 AdaBoost AUC:   0.500776
6           Tuned AdaBoost AUC:   0.493854
14          Tuned CatBoost AUC:   0.484105
0      Logistic Regression AUC:   0.482218
